# Computer vision

## Capture and save all frames from a video with OpenCV

Run `pip install opencv-python` to install opencv-python, and save a video file to the project directory as `test.MOV`. You can then run the following code to capture and save all frames from the video:

In [3]:
import cv2
import os

# Create the "frames" folder if it doesn't exist
if not os.path.exists("frames"):
    os.mkdir("frames")

# Open video file (replace with your video path)
cap = cv2.VideoCapture("test.MOV")

# Frame counter
count = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Save frame as PNG image
    filename = f"frames/frame_{count:04d}.png"  # Pad with zeros (e.g., frame_0001.png)
    cv2.imwrite(filename, frame)

    count += 1

cap.release()
cv2.destroyAllWindows()

Alternatively, here's how to capture and save a snap from your web cam:

In [9]:
import cv2
import os

# Capture a frame from the camera
cap = cv2.VideoCapture(0)  # Open the default camera
ret, frame = cap.read()    # Read a frame from the camera
cap.release()              # Release the camera

if frame is None:
    print("Failed to load image")

# Specify the folder name where you want to save the frame
folder_name = 'frames'

# Check if the folder exists, if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Specify the path and name of the file you want to save
# For example, saving the image as frame1.png in the frames folder
file_name = os.path.join(folder_name, 'frame1.png')

# Save the frame as a .png image
cv2.imwrite(file_name, frame)

print(f'Frame saved as {file_name}')

Frame saved as frames\frame1.png
